In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Example text data
text_data = """
            Deep learning is a subfield of artificial intelligence that focuses on developing algorithms that can learn and make predictions from data. 
            It has gained immense popularity in recent years due to its ability to solve complex problems in various domains such as computer vision, 
            natural language processing, and robotics.
            """

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
encoded_text = tokenizer.texts_to_sequences([text_data])[0]

# Generate training sequences
sequences = []
for i in range(1, len(encoded_text)):
    sequence = encoded_text[:i+1]
    sequences.append(sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

# Split sequences into input and output
X = sequences[:, :-1]
y = sequences[:, -1]

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=2)

# Generate new text
seed_text = "deep learning"
for _ in range(10):
    encoded_seed = tokenizer.texts_to_sequences([seed_text])[0]
    encoded_seed = pad_sequences([encoded_seed], maxlen=max_sequence_len-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(encoded_seed), axis=-1)
    predicted_word = [word for word, index in tokenizer.word_index.items() if index == predicted_word_index][0]
    seed_text += " " + predicted_word

print("Generated Text:", seed_text)
